<a href="https://colab.research.google.com/github/scskalicky/LING-226-vuw/blob/main/22_using_lexical_resources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing texts with lexical resources

In this notebook I have created access to the text files which I analysed in my [*Please, please, just tell me*](https://journals.uic.edu/ojs/index.php/dad/article/view/11387/10030) article. If you [watch my video](https://vstream.au.panopto.com/Panopto/Pages/Viewer.aspx?id=554acdba-a8cc-4c3b-8d00-adf500816e28), you'll have a good background/understanding of what I did in the paper and how the linguistic features were used.

Of course, I did that entire analysis in R and on my own local computer. Doing it in Colab is a bit different but here is a chance to be able to play with the same data. I'll provide you with a method to load in all the texts (this asks you to download them all to your Google Drive).

## Working with `.csv` files

I'm going to use two things we haven't discussed, but are useful. First, I'll use the `pandas` library because it has a really easy method to read in a `.csv` file from a URL. (A .csv file is a file where text is separated by commas, which can be opened as a spreadsheet in excel as well as text editors). 

In the cell below, I import `pandas` and then read in the `.csv` file. 

I then rename the file and ask for four specific columns from the file, which is similar to slicing.

In [ ]:
# You can download a .csv files which contains all texts
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/scskalicky/LING-226-vuw/main/wwdtm_cleaned3.csv')

# subset the data so that we only have the columns that we want. 
wwdtm_data = data[['date', 'Order', 'Veracity', 'clean_text']]

You can view the entire file by calling the file name. This is a pandas dataframe, which is an object type we haven't explored, but you can think of it for now as a spreadsheet. You can see that by default we get a peek at the first few and last few rows of the spreadsheet. 

The structure of the data is:
- `date` is the day of the show, and each date will have three stories
- `order` is the order of the stories presented on any one date. So, each date has order values of 1, 2, 3
- `veracity` marks if the story was truthful (1) or a lie (0)
- `clean_text` is the text of the story, which I've already cleaned to be lower case and fixed spelling mistakes. 

In [ ]:
wwdtm_data

We can slice by asking for a specific column name, which would return that entire column, but then slice just a specific text. 

In [ ]:
# look at the first text, which is a true story!
wwdtm_data['clean_text'][0]

### Creating individual text files from the `.csv`

Ok, so we *could* operate on the texts in the `.csv` file. But let's take this opportunity to see how we could convert each individual row into a text file. I'll process each row so that I create a unique filename for each text file and store the contents of `clean_text` as the contents of the `.txt` file. 

In the cell below I show you how to write this data to your Google Drive. You'll need to manually create a folder named `wwdtm` in your Google drive (or revise the path I've used in the `open()` call below). 

I first initiate a `for loop` which is equal to the length of the data, going through each row of the data, one-at-a-time. (There are other ways to do this which are more computationally efficient, but we can keep it simple for now).

I then check whether the value of `Veracity` is equal to 1 or 0 (1 means true, 0 means fake). I create a temporary variable, `category`, which has the value of `true` or `fake` depending upon the value of `Veracity`. 

I then create a filename which is the first 10 characters of the date, which will be YYYY-MM-DD, plus the order of the story (1, 2, or 3), plus whether the story was truth or fake. 

I then save a file to the `wwdtm` folder which include the contents of `clean_text` saved to the filename. I manually created this folder in my `texts` folder in my drive, and you'll need to do the same to emulate this behaviour (or, again, change the filepath that I'm using). You'll also need to have mounted your google drive. 

If you run the cell below, you will add ~750 text files to a folder in your Google Drive. Make sure to create the `wwdtm` folder first, and wait at least 10 seconds before running the function. It might take your Google Drive a bit to update the folder with all the files. 

**a word of caution**

when you run this function with an empty folder, it will run quickly. If you run it with texts already created, it will be much slower - so try to set this up once and then not do it again :)

If you do want to run it again, it will honestly be faster to delete the wwdtm folder from your drive, and then create it again, and rerun the function. 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# write the data to your google drive
# make a folder in your google drive named 'wwdtm'  
# loop through each line in the csv
for row in range(len(wwdtm_data)):

  # create a true/fake category based on the veracity label
  if wwdtm_data['Veracity'][row] == 1:
    category = 'true'
  else:
    category = 'fake'
  
  # create the filename, whiich is the year/day + the order+ the category
  filename = wwdtm_data['date'][row][:10] + '_' + str(wwdtm_data['Order'][row]) + '_' + category

  # write each file to the wwdtm folder. 
  open('/content/drive/MyDrive/texts/wwdtm/' + filename + '.txt', 'w').write(wwdtm_data["clean_text"][row])

### Creating a categorised NLTK corpus

Sweet, you hopefully have access to the files now through using my method or a method of your choice. Let's now bring them into Colab in a format where we can analyse them. Specifically, I'll use the NLTK function to create a categorized corpus from the WWDTM texts. 

In the cell below, I first import the `CategorizedPlaintextCorpusReader` class, which will let me create a categorized corpus. 

I then create a variable which saves the path to my texts. 

I then create the corpus by pointing it at my corpus location. I ask the corpus to use any file with the second argument (using `'.*'`), and then tell the function to find my categories in the last four characters of the filename. That's what the regular expression is doing in `cat_pattern` - I've put the () around the last four `.` of my text filesnames, which is either `true` or `fake`.  This creates an NLTK corpus with the associated categories. 

In [ ]:
import nltk
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

corpus_location = '/content/drive/MyDrive/texts/wwdtm/'

wwdtm_corpus = CategorizedPlaintextCorpusReader(corpus_location, '.*', cat_pattern = '.*(....).txt')

In [ ]:
# categories work!
wwdtm_corpus.categories()

In [ ]:
# we can use the fileids()
wwdtm_corpus.fileids()[:10]

In [ ]:
# let's look at the words for a text in the corpus
' '.join([w for w in wwdtm_corpus.words('2017-02-04_2_fake.txt')])

We now have access to all the same functions a built-in NLTK corpus has. We could use the handy conditional frequency distribution to then compare the two categories. See below. I am using basically the same code used in [NLTK chapter 02, Section 1.3.](https://www.nltk.org/book/ch02.html) 

In [ ]:
# Create CFD for fake/true
wwdtm_cfd = nltk.ConditionalFreqDist(
    (category, word)
    for category in wwdtm_corpus.categories()
    for word in wwdtm_corpus.words(categories = category))

You can now search for target words in the CFD to see if there are any lexical differences between our corpora. I decided to see if there was a difference in the use of certain pronouns between fake and truthful stories. The results below tenatively suggest that there is a preference for first-person pronouns in the fake stories when compared to the truthful stories. We should remember that 2/3 of the corpus is fake, but even with that imbalance, it seems that truthful stories avoid first-person. Interesting! Perhaps this has something to do with creating a fictional narrative? 

In [ ]:
# You can now search for target vocab
categories = ['true', 'fake']
target_words = ['you', 'we', 'my', 'i', 'our', 'us']

In [ ]:
wwdtm_cfd.tabulate(condition = categories, samples = target_words)

We can now emulate some of the same analyses done in the original paper. I looked for a variety of differences in the text types in my analysis, including things like words related to strength from the General Inquirer list. Let's load that resource in. 


In [ ]:
# GI list
import requests

gi_url = 'https://raw.githubusercontent.com/scskalicky/LING-226-vuw/main/lexical%20resources/inquirerbasic.txt'
raw_gi = requests.get(gi_url).text.split('\n')

In [ ]:
# turn the list into a dictionary we can use. 
gi_dict = dict()

for category in raw_gi:
  gi_dict[category.split('\t')[0]] = category.split('\t')[1:]

In [ ]:
# we can see the categories here, let's get the STRONG_GI words into a list. 
gi_dict.keys()

In [ ]:
strong_words = gi_dict["Strong_GI"]

There are a lot of words in this list and some repeats (which might be my fault?!). Let's reduce the list just so it is more manageable for this analysis.  

In [ ]:
# full list is 1902 long
len(strong_words)

In [ ]:
# with ~400 repeats
len(set(strong_words))

In [ ]:
small_strong = set([w for w in strong_words if len(w) < 6])

In [ ]:
# much more managable
len(small_strong)

In my results for the published paper I said that fake texts have more strong words (limited to adjectives, actually, but let's ignore that for now). Check out these results, do they match with the results of the please, please paper?  (e.g., do the fake stories have more "strength" words than truth?). Compare the orange to blue lines below ( you can also run `.tabulate()` to compare the numbers manually. 

In [ ]:
# import matplotlib so we can adjust the plot size
import matplotlib.pyplot as plt
plt.figure(figsize = (20, 10))

# plot the CFD for strength words
wwdtm_cfd.plot(conditions = categories, samples = small_strong)

## Operating outside of NLTK functions

So that's how we could use the built-in NLTK stuff. We can also just say "screw you NLTK" and do our own custom stuff. Let's do that below. I wanna use VADER and analyse the average compound sentiment for my texts. How can I do that? First, let's import the vader stuff. It now gives a fun warning about twython which we can ignore. 


In [ ]:
# Import the vader sentiment analyzer and save to the variable `sid`

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

Ok, I'm going to make a function named `vader_parser()`. This function is going to read in my texts from the `wwdtm` directory and analyse them one at a time. 

I first create two empty output lists, one for truthful, and one for deceptive stories.

I then use `glob` to grab a list of all the `.txt` files in my `wwdtm` directory. 

I then initiate a `for loop` over the filepaths. For each filepath, I read in the file. I then check whether the file name contains `true` or `fake` in the four characters before `.txt` in the filename — notice the slice to do so. 

I then run the VADER sentiment score on the file and append the result to `true_polarity` or `fake_polarity` according to the filename. 

Once the loop is done, I gather the average of each list and print the results. 

In [ ]:
import glob

def vader_parser():

  # initalise empty lists for outout
  true_polarity = []
  fake_polarity = []

  # grab list of texts in the dir
  text_paths = glob.glob('/content/drive/MyDrive/texts/wwdtm/*')
  
  # loop through the texts
  for path in text_paths:

    # this is where the text is actually opened
    t = open(path).read()

    # check if text is true or false
    # the calculate vader score
    # note I slice out the `compound` value from the result. 
    if path[-8:-4] == 'true':
      sentiment_compound = sid.polarity_scores(t)['compound'] # slice out just the compound score
      true_polarity.append(sentiment_compound)
    else:
      sentiment_compound = sid.polarity_scores(t)['compound']
      fake_polarity.append(sentiment_compound)

  # once the loop is done, calculate the avg compound scores for each text categorie
  avg_true_sent = sum(true_polarity)/len(true_polarity)
  avg_fake_sent = sum(fake_polarity)/len(fake_polarity)

  # print the results
  print(f'true polarity = {avg_true_sent}')
  print(f'fake polarity = {avg_fake_sent}')

Let's run the function. As you can see, the fake stories have, on average, a higher positive sentiment. This might suggest that the authors of the deceptive stories are consciously or subconsciously using language in a specific way, which manifests as more positive sentiment. This also matches the results in the *please, please* paper :)

In [ ]:
# run the function
vader_parser()

You could adapt the above function to calculate a variety of features per text, rather than writing different functions. One thing to reflect upon is whether you might want to tokenize and pos_tag first so that you could run the results on specific word categories or even phrases.

I hope that this has shown you that you have a pretty wide mandate on how you might want to analyse texts for this course. Although NLTK might seem like you have to use it a certain way, you're free to leave or take any aspects of it you'd like to use. You can also buck NLTK and do you own custom analysis, like I did (although technically I used the NLTK class for VADER sentiment!). 

For the remainder of this workshop, please take the time to continue analysing these texts *or* reading in your own texts. You should try to find ways to use some of the other lexical resources I've provided, as well as anything else you might want to anlayse. The goal is to become more comfortable comparing texts for various linguistic feat properties. 